# <h2> 4. Modeling (Part 2) </h2>

First, I'm going to load the data set and then import necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import time

#Define a timer
def print_time(start_time, end_time):
    elapsed_time = end_time - start_time
    hours = elapsed_time // 3600
    mins = (elapsed_time - hours*3600)//60
    secs = (elapsed_time - hours*3600 - mins*60) // 1
    
    return print("\nTime elapsed: {} hours {} minutes and {} seconds".format(hours, mins, secs))

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Load data sets
X_train = pd.read_csv("saved_files/concrete_x_train.csv")
X_test = pd.read_csv("saved_files/concrete_x_test.csv")
y_train = pd.read_csv("saved_files/concrete_y_train.csv")
y_test = pd.read_csv("saved_files/concrete_y_test.csv")
results = pd.read_csv("saved_files/concrete_model_results.csv", index_col = 0)

#Check loaded data sets
print("X_train")
print(X_train.head(), end = "\n\n")
print("X_test")
print(X_test.head(), end = "\n\n")
print("y_train")
print(y_train.head(), end = "\n\n")
print("y_test")
print(y_test.head(), end = "\n\n")
print("Results")
print(results, end = "\n\n")

X_train
   cement   slag    ash  water  superplasticizer  coarse_agg  fine_agg  age
0   190.0  190.0    0.0  228.0               0.0       932.0     670.0  365
1   388.6   97.1    0.0  157.9              12.1       852.1     925.7   28
2   192.0  288.0    0.0  192.0               0.0       929.8     716.1    7
3   164.0    0.0  200.0  181.0              13.0       849.0     846.0   28
4   379.5  151.2    0.0  153.9              15.9      1134.3     605.0   91

X_test
   cement   slag     ash   water  superplasticizer  coarse_agg  fine_agg  age
0  239.60  359.4    0.00  185.70              0.00       941.6    664.30   28
1  122.60  183.9    0.00  203.50              0.00       958.2    800.10    3
2  491.00   26.0  123.00  201.00              3.93       822.0    699.00   28
3  190.34    0.0  125.18  161.85              9.88      1088.1    802.59  100
4  304.00  140.0    0.00  214.00              6.00       895.0    722.00   28

y_train
    strength
0  53.692254
1  50.697170
2  21.480625

In [3]:
#import models & other required packages
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

#Import libraries for models
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#! pip install xgboost #uncomment if xgboost is not already installed 
from xgboost import XGBRegressor

# install and import libraries for hyperparameters tuning
#! pip install scikit-optimize #uncomment if scikit-optimize is not already installed
from skopt import BayesSearchCV
from skopt import space

## <h3> 4.4. Random Forest</h3>

Now, I will use a Random Forest model. Although standardization is not necessarily useful for this model, I will define a pipe that will standardize the data before fitting the model for consistency.


In [4]:
#define Random Forest pipe
pipe_rf = Pipeline([("scaler", StandardScaler()),
                     ("model", RandomForestRegressor(random_state = 10, 
                                                     criterion = "mse"))
                        
])

In [5]:
#fit Random Forest pipe
start_time = time.perf_counter()

pipe_rf.fit(X_train, y_train)

end_time = time.perf_counter()
print_time(start_time, end_time)


Time elapsed: 0.0 hours 0.0 minutes and 0.0 seconds


In [6]:
#Predict using fitted Random Forest model
rf_preds = pipe_rf.predict(X_test)

rf_rmse = mean_squared_error(y_test, rf_preds, squared = False)
print("Random Forest RMSE:",rf_rmse)

Random Forest RMSE: 4.71044186796703


In [7]:
#Store results
results.loc["Random Forest"] = rf_rmse
results

,RMSE
Linear Regression,10.659515
KNN,8.557282
KNN + GridSearchCV,7.384883
Random Forest,4.710442


With the Random Forest model, the RMSE is **4.71**, a major improvement over previous models.

## <h3> 4.5. Random Forest with GridSearchCV</h3>

Now, I will use the cross validation and tune hyperparameters for the Random Forest model to see if we have an RMSE improvement.
The hyperparameters I will tune are:


*   Number of trees in the forest (i.e. *n_estimators*)
*   Number of features to consider when looking for the best split (i.e. *max_features*)



In [8]:
#Define Random Forest + GridSearchCV pipe
pipe_rf_grid = Pipeline([("scaler", StandardScaler()),
                          ("model", RandomForestRegressor(random_state = 10, criterion = "mse"))
                        
])
#Define parameter space
params_rf = {'model__n_estimators': range(100, 1001, 50), #from 100 to 1000 in increments of 50
              'model__max_features': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
           }


#use n_jobs = -1 to use all processors
rf_grid = GridSearchCV(pipe_rf_grid, param_grid = params_rf, n_jobs = -1, cv=10, scoring='neg_root_mean_squared_error')

In [9]:
#Fit Random Forest + GridSearchCV
start_time = time.perf_counter()

rf_grid.fit(X_train, y_train)

end_time = time.perf_counter()
print_time(start_time, end_time)


Time elapsed: 0.0 hours 8.0 minutes and 50.0 seconds


In [10]:
#Print best parameters
print(rf_grid.best_params_)

{'model__max_features': 0.5, 'model__n_estimators': 950}


In [11]:
#Predict using fitted Random Forest + GridSearchCV model
rf_grid_preds = rf_grid.best_estimator_.predict(X_test)

rf_grid_rmse = mean_squared_error(y_test, rf_grid_preds, squared = False)
print("Random Forest + GridSearchCV RMSE:",rf_grid_rmse)

Random Forest + GridSearchCV RMSE: 4.504616562269336


In [12]:
#Store results
results.loc["Random Forest + GridSearchCV"] = rf_grid_rmse
results

,RMSE
Linear Regression,10.659515
KNN,8.557282
KNN + GridSearchCV,7.384883
Random Forest,4.710442
Random Forest + GridSearchCV,4.504617


In [13]:
#Store predictions for an Ensemble
predictions = pd.DataFrame({"Random Forest + GridSearchCV" : rf_grid_preds})
predictions

,Random Forest + GridSearchCV
0,38.966624
1,6.558833
2,53.567455
3,38.385727
4,38.868662
...,...
201,29.997680
202,34.475910
203,63.428361
204,46.738008


With GridSearchCV, the RMSE of the Random Forest model improves to **4.50**, an improvement over the Random Forest model with default parameters.

## <h3> 4.6. Extreme Gradient Boosting (XGBoost) </h3>

Now, I will use an Extreme Gradient Boosting (XGBoost) model. Like before, I will define a pipe that will standardize the data before fitting the model for consistency.


In [14]:
#define XGBoost pipe
pipe_xgb = Pipeline([("scaler", StandardScaler()),
                     ("model", XGBRegressor(random_state = 10))
                        
])

In [15]:
#fit XGBoost pipe
start_time = time.perf_counter()

pipe_xgb.fit(X_train, y_train)

end_time = time.perf_counter()
print_time(start_time, end_time)


Time elapsed: 0.0 hours 0.0 minutes and 0.0 seconds


In [16]:
#Predict using fitted XGBoost model
xgb_preds = pipe_xgb.predict(X_test)

xgb_rmse = mean_squared_error(y_test, xgb_preds, squared = False)
print("XGBoost RMSE:",xgb_rmse)

XGBoost RMSE: 4.328552120997665


In [17]:
#Store results
results.loc["XGBoost"] = xgb_rmse
results

,RMSE
Linear Regression,10.659515
KNN,8.557282
KNN + GridSearchCV,7.384883
Random Forest,4.710442
Random Forest + GridSearchCV,4.504617
XGBoost,4.328552


XGBoost with its default hyperparameters gives an RMSE of **4.33**.

## <h3> 4.7. Extreme Gradient Boosting (XGBoost) with BayesSearchCV</h3>

Now, I will optimize some hyperparameters of the Extreme Gradient Boosting (XGBoost) model using BayesSearchCV. BayesSearchCV from scikit-optimize library is used to conduct Bayesian optimization with cross validation.


In [18]:
#Define XGBoost + BayesSearchCV pipe
pipe_xgb_bo = Pipeline([("scaler", StandardScaler()),
                          ("model", XGBRegressor(random_state = 10,
                                                 objective = "reg:squarederror",
                                                 verbosity = 0))
                        
])
#Define parameter space
params_xgb_bo = {'model__n_estimators': space.Integer(100, 1000, 'log-uniform'),
              'model__learning_rate': space.Real(10e-6, 1.0, 'log-uniform'),
              'model__max_depth': space.Integer(0, 50, 'uniform')
           }


#use n_jobs = -1 to use all processors
xgb_bo = BayesSearchCV(pipe_xgb_bo, 
                       params_xgb_bo, 
                       n_iter=50, 
                       cv=10, 
                       random_state = 10, 
                       scoring='neg_root_mean_squared_error')

In [19]:
#Fit XGBoost + BayesSearchCV
start_time = time.perf_counter()

xgb_bo.fit(X_train, y_train)

end_time = time.perf_counter()
print_time(start_time, end_time)


Time elapsed: 0.0 hours 8.0 minutes and 5.0 seconds


In [20]:
#Print best parameters
print(xgb_bo.best_params_)

OrderedDict([('model__learning_rate', 0.08920889597332297), ('model__max_depth', 40), ('model__n_estimators', 1000)])


In [21]:
#Predict using fitted XGBoost + BayesSearchCV model
xgb_bo_preds = xgb_bo.best_estimator_.predict(X_test)

xgb_bo_rmse = mean_squared_error(y_test, xgb_bo_preds, squared = False)
print("XGBoost + BayesSearchCV RMSE:",xgb_bo_rmse)

XGBoost + BayesSearchCV RMSE: 4.13665462019597


In [22]:
#Store results
results.loc["XGBoost + BayesSearchCV"] = xgb_bo_rmse
results

,RMSE
Linear Regression,10.659515
KNN,8.557282
KNN + GridSearchCV,7.384883
Random Forest,4.710442
Random Forest + GridSearchCV,4.504617
XGBoost,4.328552
XGBoost + BayesSearchCV,4.136655


In [23]:
#Store predictions for an Ensemble
predictions["XGBoost + BayesSearchCV"] = xgb_bo_preds
predictions

,Random Forest + GridSearchCV,XGBoost + BayesSearchCV
0,38.966624,42.218987
1,6.558833,4.965951
2,53.567455,52.095085
3,38.385727,38.684082
4,38.868662,35.667572
...,...,...
201,29.997680,27.292423
202,34.475910,31.838600
203,63.428361,62.630512
204,46.738008,51.595390


XGBoost + BayesSearchCV gives an RMSE of **4.14**.

## <h3> 4.8. Ensemble </h3>

Now, I will combine the predictions made by the Random Forest + GridSearchCV model with the ones by XGBoost + BayesSearchCV to improve the results. I will take their mean for the ensemble model.

In [24]:
#Take the average of predictions made by XGBoost and Random Forest + GridSearchCV
ensemble_preds = (predictions.iloc[:, 0] + predictions.iloc[:, 1])/2
predictions["Ensemble"] = ensemble_preds
predictions

,Random Forest + GridSearchCV,XGBoost + BayesSearchCV,Ensemble
0,38.966624,42.218987,40.592805
1,6.558833,4.965951,5.762392
2,53.567455,52.095085,52.831270
3,38.385727,38.684082,38.534905
4,38.868662,35.667572,37.268117
...,...,...,...
201,29.997680,27.292423,28.645052
202,34.475910,31.838600,33.157255
203,63.428361,62.630512,63.029436
204,46.738008,51.595390,49.166699


In [25]:
ensemble_rmse = mean_squared_error(y_test, ensemble_preds, squared = False)
print("Ensemble", ensemble_rmse)

Ensemble 4.123771328786997


In [26]:
#Store results
results.loc["Ensemble"] = ensemble_rmse
results

,RMSE
Linear Regression,10.659515
KNN,8.557282
KNN + GridSearchCV,7.384883
Random Forest,4.710442
Random Forest + GridSearchCV,4.504617
XGBoost,4.328552
XGBoost + BayesSearchCV,4.136655
Ensemble,4.123771


The ensemble model using Random Forest + GridSearchCV and XGBoost + BayesSearchCV gives an RMSE of **4.12**.

Let's save the latest version of the model results and the ensemble predictions as csv files.

In [27]:
#Save model results as a csv file
results.to_csv('saved_files/concrete_model_results.csv', index=True)
#Save ensemble predictions as a csv file
predictions.to_csv('saved_files/concrete_ensemble_predictions.csv', index=False)

# <h2> 5. Conclusion </h2>

As seen the results table above, the Ensemble model of both Random Forest + GridSearchCV and XGBoost + BayesSearchCV obtained the best result with an RMSE of **4.12** while the XGBoost + BayesSearchCV was the second best model with an RMSE of **4.14**.

The Ensemble model improved the RMSE score by around **6.54** MPa (i.e. megapascals) from the base Linear Regression Model.

Next, in another notebook, I will use Neural Networks to try to obtain an even lower RMSE score. 

Overall, this data set was an interesting set.